<a href="https://colab.research.google.com/github/zohyan/Understanding-PyTorch/blob/master/Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Neural networks can be constructed using the **torch.nn** package. **nn** depends on **autograd** to define models and differentiate them. An **nn.Module** contains layers, and a method **forward(input)** that returns the **output**.

For example, look at this network that classifies digit images:

![alt text](https://drive.google.com/uc?id=1gh0Ew2t435IZ0NuUR1Mnl75wDYvkPkyL)

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:


1.   Define the neural network that has some learnable parameters (or weights)
2.   Iterate over a dataset of inputs
3.   Process input through the network
4.   Compute the loss (how far is the output from being correct)
5.   Propagate gradients back into the network’s parameters
6.   Update the weights of the network, typically using a simple update rule: **weight = weight - learning_rate * gradient**

### 1. Define the network

Let’s define this network:

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        # kernel
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    # this is where we use the torch.nn.functional package
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), (2, 2))
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the **forward** function, and the **backward** function (where gradients are computed) is automatically defined for you using **autograd**. You can use any of the Tensor operations in the **forward** function.

The **learnable parameters** of a model are returned by **net.parameters()**

In [0]:
params = list(net.parameters())
print(len(params))
print(params[0].size()) 

10
torch.Size([6, 1, 3, 3])


Let’s try a random 32x32 input. Note: expected input size of this net (LeNet) is 32x32. To use this net on the MNIST dataset, please resize the images from the dataset to 32x32.

In [0]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0350, -0.1174, -0.0158,  0.1175, -0.0825,  0.1636,  0.0283, -0.0860,
         -0.0282,  0.0490]], grad_fn=<AddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [0]:
# Calculation of the gradients of a randomly
# generated tensor via backpropagation

net.zero_grad()
out.backward(torch.randn(1, 10))

**torch.nn only supports mini-batches**. The entire **torch.nn** package only supports inputs that are a mini-batch of samples, and not a single sample.

For example, **nn.Conv2d** will take in a 4D Tensor of **nSamples x nChannels x Height x Width**.

If you have a single sample, just use **input.unsqueeze(0)** to add a fake batch dimension.

## **2. Loss Function**

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package . A simple loss is: **nn.MSELoss** which computes the mean-squared error between the input and the target.

In [0]:
output = net(input)
target = torch.randn(10)  # a dummy target, for example
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(0.7575, grad_fn=<MseLossBackward>)


So, when we call **loss.backward()**, the whole graph is **differentiated w.r.t. the loss**, and all Tensors in the graph that has **requires_grad=True** will have their **.grad** Tensor accumulated with the gradient.

## **3.   Backprop**



To backpropagate the error all we have to do is to **loss.backward()**. **You need to clear the existing gradients though, else gradients will be accumulated to existing gradients.**

Now we shall call loss.backward(), and have a look at conv1’s bias gradients before and after the backward.

In [0]:
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
tensor([0., 0., 0., 0., 0., 0.])
conv1.bias.grad after backward
tensor([ 0.0143,  0.0050, -0.0081, -0.0111,  0.0109,  0.0094])


## **4.   Update the weights**

However, as you use neural networks, you want to use various different update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc. To enable this, we built a small package: **torch.optim** that implements all these methods. Using it is very simple:

In [0]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update